In [ ]:
!pip uninstall -y msspackages

In [ ]:
!pip install /root/msspackages/dist/msspackages-0.0.7-py3-none-any.whl

In [ ]:
from msspackages import setup_runner
setup_runner()

In [ ]:
!python3 -m pip install -r requirements.txt

In [ ]:
time_steps = 20
batch_size = 32
train_n_samples = batch_size * 1000
test_n_samples = round((batch_size*1000*0.2) / 0.8)

In [ ]:
features = ['pod_cpu_utilization','pod_memory_utilization','pod_network_total_bytes']
scaled_features = ['scaled_pod_cpu_utilization','scaled_pod_memory_utilization','scaled_pod_network_total_bytes']

In [ ]:
import pandas as pd
pod_training_df = pd.read_parquet('training_pod_2022_9_9.parquet', engine='fastparquet')
pod_testing_df = pd.read_parquet('testing_pod_2022_9_9.parquet', engine='fastparquet')


In [ ]:
pod_training_df.shape

In [ ]:
pod_testing_df.shape

In [ ]:
pod_training_df.head()

In [ ]:
pod_training_df['freq'] = pod_training_df.groupby('pod_id')['pod_id'].transform('count')

In [ ]:
filtered_pod_df = pod_training_df[pod_training_df["freq"] > 20]

In [ ]:
filtered_pod_df.head()

In [ ]:
selected_pod_list = filtered_pod_df['pod_id'].sample(train_n_samples).to_list()

In [ ]:
len(selected_pod_list)

In [ ]:
from sklearn.preprocessing import StandardScaler

def pod_tensor_builder(pod_id, input_pod_processed_df, time_steps, scaled_features, features):

    ##pick random df, and normalize
    print(pod_id)
    pod_fe_df = input_pod_processed_df.loc[(input_pod_processed_df["pod_id"] == pod_id)]
    pod_fe_df = pod_fe_df.sort_values(by='Timestamp').reset_index(drop=True)
    pod_fe_df_len = len(pod_fe_df)
    #smaller df check 
    # if pod_fe_df_len > time_steps:
    #     print(f'Exception occurred: pod_fe_df_len-time_steps = {pod_fe_df_len-time_steps}')
    #     break

    #tensor builder
    start = random.choice(range(pod_fe_df_len-time_steps))
    pod_fe_df = pod_fe_df[start:start+time_steps]

    #scaler transformations
    scaler = StandardScaler()
    pod_fe_df[scaled_features] = scaler.fit_transform(pod_fe_df[features])

    return pod_fe_df

In [ ]:
%%time
from joblib import Parallel, delayed
import random
import multiprocessing

num_cores = multiprocessing.cpu_count()
print(num_cores)
pod_fe_df_list = Parallel(n_jobs=num_cores, prefer="processes")(delayed(pod_tensor_builder)(random_pod_id, filtered_pod_df, time_steps, scaled_features, features) for random_pod_id in selected_pod_list)

In [ ]:
pod_fe_df_list

In [ ]:
from eks_ml_pipeline import write_tensor, awswrangler_pandas_dataframe_to_s3

In [ ]:
awswrangler_pandas_dataframe_to_s3(pod_training_data,  "dish-5g.core.pd.g.dp.eks.logs.e", "pod_autoencoder_ad", "v0.0.2", "training_2022_9_9")

In [ ]:
write_tensor(pod_training_tensor, "dish-5g.core.pd.g.dp.eks.logs.e", "pod_autoencoder_ad", "v0.0.2", "training_2022_9_9")